# URBA6004 Tutorial - Extract and analyse street network with OSMnx

**Author**: Kenneth Wong (khwong12@hku.hk)

**Last Edited**: 2022-03-22

---

## Overview

In this tutorial, you will use OSMnx to:

- Extract street network from OpenStreetMap
- Compute basic summary statistics of the street networks
- Inspect betweenness centrality
- Save the street network as spatial data file for opening it in other GIS Applications

This tutorial is based on [example notebooks](https://github.com/gboeing/osmnx-examples) by Geoff Boeing, the author of the OSMnx package.

----

In [ ]:
# Install packages if you have not done so

# Install via conda in local anaconda environment
# !conda install osmnx

# Or, use pip if you are using Google Colab
# !pip install osmnx

In [ ]:
import geopandas as gpd

import networkx as nx
import osmnx as ox
import pandas as pd

%matplotlib inline
ox.__version__

---

## Introduction

### What is OSMnx?

OSMnx is a Python package that lets you **download geospatial data from OpenStreetMap** and model, project, **visualize, and analyze real-world street networks** and any other geospatial geometries. You can download and model walkable, drivable, or bikeable urban networks with a single line of Python code then easily analyze and visualize them. You can just as easily download and work with other infrastructure types, amenities/points of interest, building footprints, elevation data, street bearings/orientations, and speed/travel time.

Check the following journal article for details.

Boeing, G. 2017. OSMnx: New Methods for Acquiring, Constructing, Analyzing, and Visualizing Complex Street Networks. Computers, Environment and Urban Systems 65, 126-139. doi:10.1016/j.compenvurbsys.2017.05.004

*(excrepeted from [OSMnx documnetation](https://osmnx.readthedocs.io/en/stable/))*

### What is OpenStreetMap (OSM)?

OpenStreetMap (OSM) is a digital map database of the world built through crowdsourced volunteered geographic information (VGI). OSM is supported by the nonprofit OpenStreetMap Foundation. **The data from OSM is freely available for visualization, query and download**.

OSM works in a style similar to Wikipedia, in which virtually all features are open to editing. Since its establishment in 2004, this map database has matured enough in some locations to the point where its detail and precision rival "authoritative" datasets from governments and commercial entities.

### Data available via OSM

Same as other spatial data, the 2 essential ingredients of OSM data are **geometry** and **descriptive attributes**.

To contribute a feature to OSM, you typically digitize a geometry (a point, line, or area) and then add descriptive attributes, or tags. For example, to tag a grocery store, you trace its building footprint and tag it with `shop=supermarket`. There's no restriction on the tags you can use, but the data is only useful to the degree that you tag things consistent with the way other OSM users have applied the tags.

To promote consistency in tagging, the OSM community has an informal tag voting and approval process organized on the [OpenStreetMap wiki](https://wiki.openstreetmap.org/wiki/Main_Page) site. Approved tags are added to the online documentation so that others can easily find and apply them. For example, the tag `shop=supermarket` denotes a grocery store (see the [Wiki page of shop=supermarket](https://wiki.openstreetmap.org/wiki/Tag:shop=supermarket)). Before you add a tag, check the wiki to make sure that you're using the established tag and syntax.

*(excrepeted from [OpenStreetMap and its use as open data Chapter of Open Web Mapping Course by PennState](https://www.e-education.psu.edu/geog585/node/738))*

---


## Part I: Extract street networks

There are multiple way to specify the "study area" for extracting street networks in OSMnx, including:

- pass a bounding box
- pass a lat-lng point and bounding box distance in meters
- pass a lat-lng point and network distance in meters
- pass an address and distance (bounding box or network) in meters
- pass a spatial polygon

Here, we will look into two functions in OSMnx to extract street network, `graph_from_point` and `graph_from_polygon`.

### Extract street network from a point with provided distance distance

A quick way to extract street network is to first provide a point on earth (in latitude/longitude), then provide a euclidean distance. This generates a rectangular boundary (*bounding box* (*bbox*) in jargon), and OSMnx will use this boundary to "clip" the street network from OSM.

The `graph_from_point` takes the following arguments:

- **center_point**: the (lat, lng) center point around which to construct the graph
- **dist**: area to be covered from the center of the graph, in meters
- **network_type**: what type of street network to get. can be one of the following:
    - ‘drive’ – get drivable public streets (but not service roads)
    - ‘drive_service’ – get drivable public streets, including service roads
    - ‘walk’ – get all streets and paths that pedestrians can use (this network type ignores one-way directionality)
    - ‘bike’ – get all streets and paths that cyclists can use
    - ‘all’ – download all (non-private) OSM streets and paths
    - ‘all_private’ – download all OSM streets and paths, including private-access ones

In [ ]:
# extract the street network of:
# 1000 meters from the point (22.3192, 114.1693), with all OSM streets and paths included
hk_graph = ox.graph_from_point(center_point = (22.3192, 114.1693), dist = 1000, network_type='all_private')

How does the street network looks like? We could use the `plot_graph` function to plot the street network.

In [ ]:
ox.plot_graph(hk_graph, node_color="w", node_size=5)

> **Question**: Do you recongise where is it?

### Extract street network from a boundary polygon

For urban analytics and projects, a more common workflow is that you have already given the study boundary of the project. You are only interested in the road network within the boundary. In this case, we could pass a spatial polygon (in the data tpye of `shapely.geometry.Polygon`) to OSMnx and ask the program to return the street network inside that polygon.

The code cell below reads the prefecture boundaries of Tokyo Prefecture and filter the dataset to include only Chiyoda District (千代田区).

In [ ]:
# boundary of Tokyo Prefecture, originally available at https://nlftp.mlit.go.jp/ksj/
BOUNDARY_URL = "https://raw.githubusercontent.com/smartnews-smri/japan-topography/main/data/municipality/geojson/s0010/N03-21_13_210101.json"

tokyo_pre_boundary = gpd.read_file(BOUNDARY_URL)

In [ ]:
tokyo_pre_boundary

In [ ]:
# filter the data to select chiyoda district only
chiyoda = tokyo_pre_boundary[tokyo_pre_boundary.N03_004 == "千代田区"]

In [ ]:
# plot the geometry with geopandas
chiyoda.plot()

In [ ]:
# plot the shapely geometry
chiyoda['geometry'][0]

In [ ]:
# remember chiyoda is a GeoDataFrame, yet what we need is the "shape" of the boundary
# therefore, we need to extract the geoemtry of the boundary polygon with ['geometry'][0]
chiyoda_graph = ox.graph_from_polygon(chiyoda['geometry'][0])

In [ ]:
ox.plot_graph(chiyoda_graph)

In addition to `plot_graph`, you could also plot the street network on an interactive map with **folium** with `plot_graph_folium`.

In [ ]:
chiyoda_folium = ox.plot_graph_folium(chiyoda_graph, popup_attribute="name", weight=2, color="#999999")

chiyoda_folium

### Graph from geocoding a place name

It is possible to pass a place name and let geocoding services to search for the "boundary" of the specified place with `graph_from_place`. 

However, I would **NOT** recommend this method as the boundary of certain places are not well-defined in OSM. Say, when you provide the name "Tsuen Wan" to the geocoder, is it returning the boundary of Tsuen Wan *Town Centre*, Tsuen Wan *New Town* or Tsuen Wan *District*? You will not know the actual boundary polygon Python will be using only after the code runs.

**It is always better to know how the polygon exactly looks like before you use it to "clip" the street network.** Otherwise, unexpected errors may occur.

In [ ]:
ty_graph = ox.graph_from_place("Tsing Yi, Hong Kong")

---

## Part II: Compute basic street network stats


### Compute summary statistics of the network

In general, we could compute 2 types of statistics of street networks - topological and geometric.

The `basic_stats` function in OSMnx computes some fundemental statistics of the street network. The function returns a dictionary with various keys indicating a type of summary statistics.

In [ ]:
stats = ox.basic_stats(hk_graph)

In [ ]:
pd.Series(stats)

# You could also "prettify" the output table by transforming it to a dataframe
# pd.DataFrame(pd.Series(stats, name="value")).round(3)

Following are explanations of some key statistics.

- `n`: count of nodes in graph
- `m`: count of edges in graph
- `k_avg` - graph's average node degree (in-degree and out-degree)
- `edge_length_total`: total edge length
- `circuity_avg`: street circuity using edges of undirected graph. Circuity is the sum of edge lengths divided by the sum of straight-line distances between edge endpoints. Calculates straight-line distance as euclidean distance if projected or great-circle distance if unprojected.
- `intersection_count`: intersections in a graph. Intersections are defined as nodes with at least min_streets number of streets incident on them.
- `self_loop_proportion`: percent of edges that are self-loops in a graph. A self-loop is defined as an edge from node u to node v where u==v.
- `street_segment_count`: street segments in a graph

<br>

> **Question**: How many nodes are there in the extracted street network?

<br>

Remember, when in doubt, check the helping page of the function with `help(ox.basic_stats)`.

**NOTE**: You may wonder the differences between `edge_length` and `street_length`. Simply speaking, the network model of street network is using *directed* graphs, and bidirectional streets are represented by 2 reciprocally directed edges. Total street length uses an undirected version of network model to quantify how many **linear meters of physical street** exist in the study area. If every street in the network is bidirectional, total edge length will be approximately twice as large as total street length (as explained by the [OSMnx author](https://github.com/gboeing/osmnx/issues/220#issuecomment-441721992)).

Streets/intersection counts and proportions are nested dicts inside the stats dict. To convert these stats to a pandas dataframe (to compare/analyze multiple networks against each other), just unpack these nested dicts first:

In [ ]:
# create a copy of the `stats` dictionary
from copy import deepcopy
stats_unpack = deepcopy(stats)

# unpack dicts into individiual keys:values
for k, count in stats_unpack["streets_per_node_counts"].items():
    stats_unpack["{}way_int_count".format(k)] = count
for k, proportion in stats_unpack["streets_per_node_proportions"].items():
    stats_unpack["{}way_int_prop".format(k)] = proportion

# delete the no longer needed dict elements
del stats_unpack["streets_per_node_counts"]
del stats_unpack["streets_per_node_proportions"]

# load as a pandas dataframe
pd.DataFrame(pd.Series(stats_unpack, name="value")).round(3)

> **Question**: Generally speaking, how many streets do the intersections of the extracted network (nodes) connected to?

It may be more interesting to map the nodes out in addition to showing some summary statistics. How about visualising the nodes according to how many street segments they are connected?

In [ ]:
# you could take a peek of what attributes are available for the nodes with `.nodes.data(data = True)` property
# hk_graph.nodes.data(data = True)

# set the "colour symbology"
nc_street_count = ox.plot.get_node_colors_by_attr(hk_graph, "street_count", cmap="YlGnBu")

fig, ax = ox.plot_graph(
    hk_graph,
    bgcolor="#232323",
    node_color=nc_street_count,
    node_size=5,
    node_zorder=2,
    edge_linewidth=0.2,
    edge_color="w"
)

### Inspect betweenness centrality


The `betweenness_centrality` function in NetworkX package allows you to compute betweenness centrality of the nodes in the network.

As the street network we extracted (`hk_graph`) is quite complex, it would take some time to compute the betweenness.

In [ ]:
# get a digraph of the street network graph (i.e. w/o parallel edges)
hk_graph_di = ox.get_digraph(hk_graph)

# compute betweenness
bc = nx.betweenness_centrality(hk_graph_di, weight="length")

# get the ID of node with max betweenness, as well as its betweenness value
max_node, max_bc = max(bc.items(), key=lambda x: x[1])
max_node, max_bc

In the network, the node with the highest betweenness centrality has ~12.8% (0.1289) of all shortest paths running through it. Let's highlight it in the plot:

In [ ]:
# set plotting options
# set the node color (nc) to red ("r") if the node is `max_node`, otherwise set to white ("w")
nc = ["r" if node == max_node else "w" for node in hk_graph.nodes]
# set the node size (ns) to 80 if the node is `max_node`, otherwise set to 5
ns = [80 if node == max_node else 5 for node in hk_graph.nodes]

# plot the street network with the plotting options above
fig, ax = ox.plot_graph(hk_graph, node_size=ns, node_color=nc, node_zorder=2)

~12.8% of all shortest paths from any node to any node runs through the node highlighted in red.

> **Question**: where is that node located?

What about the centrality of other nodes? Let's look at the relative betweenness centrality of every node in the graph.


In [ ]:
# add the betweenness centraliy values as new node attributes
nx.set_node_attributes(hk_graph, bc, "bc")

# plot the street network
nc = ox.plot.get_node_colors_by_attr(hk_graph, "bc", cmap="Reds")
fig, ax = ox.plot_graph(
    hk_graph,
    bgcolor="#232323",
    node_color=nc,
    node_size=5,
    node_zorder=2,
    edge_linewidth=0.2,
    edge_color="w"
)

---

## Part III: Save the network

Quite often, you may want to save the network you created and then make some edits in ArcGIS, QGIS, or other softwares.

### Export as geopackage

In [ ]:
ox.save_graph_geopackage(hk_graph, filepath="./data-output/hk_graph.gpkg")

### Export as shapefile

The shapefile format is proprietary and outdated (see http://switchfromshapefile.org/). Whenever possible, you should use the superior GeoPackage file format instead via the save_graph_geopackage function.
 
In most cases, the reason you need shapefile is because it is most widely supported format in existing software packages.

In [ ]:
ox.save_graph_shapefile(hk_graph, filepath="./data-output/hk_graph/")

### Export as SVG

If you are using the network to draw some masterplan-like maps, you could export the network as a SVG file and open it in AI.

In [ ]:
# save street network as SVG
fig, ax = ox.plot_graph(hk_graph, show=False, save=True, close=True, filepath="./hk_graph.svg")

---

## References

- https://github.com/gboeing/osmnx-examples/blob/main/notebooks/01-overview-osmnx.ipynb
- https://github.com/gboeing/osmnx-examples/blob/main/notebooks/03-graph-place-queries.ipynb
- https://github.com/gboeing/osmnx-examples/blob/main/notebooks/05-save-load-networks.ipynb